In [1]:
import pandas as pd

/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_zip = pd.read_csv('/Volumes/Macdrive2/DataScience/Uptake/zipCodeMarketingCosts.csv')

## Data Preparation

In [3]:
df = pd.read_csv("/Volumes/Macdrive2/DataScience/Uptake/data/train.csv", index_col='id')
#df.head()

/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['zip'] = df['zip'].replace('\-','',regex=True).astype(float)

In [5]:
new_df = df.merge(df_zip, how = 'inner', on = 'zip')

In [6]:
training_df = new_df.loc[:,['zip','cluster','major','pepstrfl','rfa_2f','rfa_2a','marketingCost','responded']]
training_df['pepstrfl'].replace(('X',' '), (1,0), inplace=True)
training_df['major'].replace(('X',' '), (1,0), inplace=True)

In [7]:
training_df['cluster'] = pd.to_numeric(training_df['cluster'], errors='coerce')
training_df = pd.get_dummies(training_df, columns = ['rfa_2a'])
training_df['cluster'].fillna((training_df['cluster'].mean()), inplace=True)

In [8]:
training_df.describe()

,zip,cluster,major,pepstrfl,rfa_2f,marketingCost,responded,rfa_2a_D,rfa_2a_E,rfa_2a_F,rfa_2a_G
count,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000,182190.000000
mean,64259.473270,27.917281,0.003057,0.474356,1.910039,6.777915,0.050766,0.077749,0.226511,0.492486,0.203255
std,23969.472985,14.279578,0.055208,0.499343,1.073082,6.332381,0.219519,0.267776,0.418574,0.499945,0.402422
min,801.000000,1.000000,0.000000,0.000000,1.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40475.000000,16.000000,0.000000,0.000000,1.000000,2.200000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,63105.000000,27.917281,0.000000,0.000000,2.000000,3.240000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,90007.000000,40.000000,0.000000,1.000000,3.000000,12.490000,0.000000,0.000000,0.000000,1.000000,0.000000
max,99950.000000,53.000000,1.000000,1.000000,4.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
X_df = training_df.drop('responded', axis=1)
Y_df = training_df['responded']

## Standardization

In [15]:
from sklearn.preprocessing import StandardScaler
n = StandardScaler()
X_df = pd.DataFrame(n.fit_transform(X_df), columns = [X_df.columns])

## Modelling

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_df,Y_df, test_size=0.2, random_state=0) 

### Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegressionCV
cls_lr_cv = LogisticRegressionCV(cv = 10)

cls_lr_cv.fit(X_train,Y_train)
pred_lr_cv = cls_lr_cv.predict(X_test)
cls_lr_cv.score(X_test, Y_test)

0.9491464954168725

### Decision Tree Classifier

In [18]:
from sklearn.tree import DecisionTreeClassifier

cls_dt = DecisionTreeClassifier()
cls_dt.fit(X_train,Y_train)
pred_dt = cls_dt.predict(X_test)
cls_dt.score(X_test,Y_test)

0.9689335309292497

### Random Forest Classifier

In [19]:
from sklearn.ensemble import RandomForestClassifier

cls_rf = RandomForestClassifier(n_estimators = 30)
cls_rf.fit(X_train,Y_train)
pred_rf = cls_rf.predict(X_test)
cls_rf.score(X_test,Y_test)

/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.9751358471924914

### Gradient Boosting Classifier

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

cls_gb = GradientBoostingClassifier()
cls_gb.fit(X_train,Y_train)
pred_gb = cls_gb.predict(X_test)
cls_gb.score(X_test,Y_test)

0.9491190515396015

### Support Vector Classifier

In [22]:
from sklearn.svm import SVC

cls_sv = SVC()
cls_sv.fit(X_train,Y_train)
pred_sv = cls_sv.predict(X_test)
cls_sv.score(X_test,Y_test)

0.9491464954168725

## Testing

In [23]:
test_df = pd.read_csv("/Volumes/Macdrive2/DataScience/Uptake/data/test.csv", index_col='id')

/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
test_df['zip'] = test_df['zip'].replace('\-','',regex=True).astype(float)

In [25]:
test_df = test_df.merge(df_zip, how = 'left', on = 'zip')

In [26]:
testing_df = test_df.loc[:,['zip','cluster','major','pepstrfl','rfa_2f','rfa_2a','marketingCost']]

In [27]:
testing_df['pepstrfl'].replace(('X',' '), (1,0), inplace=True)
testing_df['major'].replace(('X',' '), (1,0), inplace=True)

In [28]:
testing_df['cluster'] = pd.to_numeric(testing_df['cluster'], errors='coerce')
testing_df = pd.get_dummies(testing_df, columns = ['rfa_2a'])

In [29]:
testing_df['cluster'].fillna((testing_df['cluster'].mean()), inplace=True)

In [30]:
predicted_test = cls_rf.predict(testing_df)
print(predicted_test.shape)

(9589,)


In [31]:
final_test_df = pd.read_csv("/Volumes/Macdrive2/DataScience/Uptake/data/test.csv", index_col='id')

/Users/vishalmishra/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
final_test_df['market'] = predicted_test

In [33]:
final_test_df['market']

id
182190    0
182191    0
182192    0
182193    0
182194    0
182195    0
182196    0
182197    0
182198    0
182199    0
182200    0
182201    0
182202    0
182203    0
182204    0
182205    0
182206    0
182207    0
182208    0
182209    0
182210    0
182211    0
182212    0
182213    0
182214    0
182215    0
182216    0
182217    0
182218    0
182219    0
         ..
191749    0
191750    1
191751    0
191752    0
191753    0
191754    0
191755    0
191756    0
191757    1
191758    0
191759    0
191760    0
191761    0
191762    0
191763    0
191764    0
191765    0
191766    1
191767    0
191768    0
191769    0
191770    0
191771    1
191772    0
191773    0
191774    0
191775    0
191776    0
191777    0
191778    0
Name: market, Length: 9589, dtype: int64

In [34]:
final_test_df.to_csv('test.csv')